In [1]:
import json
from tqdm import tqdm
import torch
import time
with open('w2id++.json', 'r') as f:
    w2id = json.load(f)
with open('id2w++.json', 'r') as f:
    id2w = json.load(f)
w2id['，'] = len(id2w)
id2w.append('，')
w2id['。'] = len(id2w)
id2w.append('。')
data_list = []
with open('data_splited++.jl', 'r') as f:
    for l in f:
        d = ','.join(json.loads(l)) + '。'
        data_list.append(d)
        
batch_size = 32
data_workers = 4
learning_rate = 1e-6
gradient_accumulation_steps = 1
max_train_epochs = 3
warmup_proportion = 0.05
weight_decay=0.01
max_grad_norm=1.0

cur_time = time.strftime("%Y-%m-%d_%H:%M:%S")
device = torch.device('cuda')

In [2]:
data_list[0]

'白狐向月号山风,秋寒扫云留碧空。'

In [3]:
set([len(x) for x in data_list])

{12, 14, 16, 18, 20}

In [4]:
dlx = [[] for _ in range(5)]
for d in data_list:
    dlx[len(d) // 2- 6].append(d)

In [5]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('./GPT2').to(device)
from tokenizations import tokenization_bert_word_level as tokenization_bert
tokenizer = tokenization_bert.BertTokenizer(vocab_file="GPT2-Chinese/cache/vocab.txt")

Model loaded succeed


In [6]:
import time
import torch
time.clock = time.perf_counter
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits
def generate(model, context, length, temperature=1.0, top_k=30, top_p=0.0, device='cpu'):
    inputs = torch.LongTensor(context).view(1, -1).to(device)
    if len(context) > 1:
        _, past = model(inputs[:, :-1], None)[:2]
        prev = inputs[:, -1].view(1, -1)
    else:
        past = None
        prev = inputs
    generate = [] + context
    with torch.no_grad():
        for i in range(length):
            output = model(prev, past)
            output, past = output[:2]
            output = output[-1].squeeze(0) / temperature
            filtered_logits = top_k_top_p_filtering(output, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(torch.softmax(filtered_logits, dim=-1), num_samples=1)
            generate.append(next_token.item())
            prev = next_token.view(1, 1)
    return generate
def is_word(word):
    for item in list(word):
        if item not in 'qwertyuiopasdfghjklzxcvbnm':
            return False
    return True
def get_next(s, temperature=1,topk=10, topp = 0, device='cuda'):
    context_tokens = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(s))
    out = generate(
        model, 
        context_tokens, 
        len(s), 
        temperature, 
        top_k=topk, 
        top_p=topp, 
        device=device
    )
    text = tokenizer.convert_ids_to_tokens(out)
    for i, item in enumerate(text[:-1]):  # 确保英文前后有空格
        if is_word(item) and is_word(text[i + 1]):
            text[i] = item + ' '
    for i, item in enumerate(text):
        if item == '[MASK]':
            text[i] = ''
        elif item == '[CLS]':
            text[i] = '\n\n'
        elif item == '[SEP]':
            text[i] = '\n'
    text = ''.join(text).replace('##', '').strip()
    return text

def print_cases():
    print(get_next('好好学习，') + '\n')
    print(get_next('白日依山尽，') + '\n')
    print(get_next('学而时习之，') + '\n')
    print(get_next('人之初性本善，') + '\n')
print_cases()

好好学习，精义苦难酬

白日依山尽，儿今无肺士。

学而时习之，健还祭。一全

人之初性本善，以文渊源驰。君



In [7]:
class MyDataSet(torch.utils.data.Dataset):
    def __init__(self, examples):
        self.examples = examples
    def __len__(self):
        return len(self.examples)
    def __getitem__(self, index):
        example = self.examples[index]
        return example, index

def the_collate_fn(batch):  
    indexs = [b[1] for b in batch]
    r = tokenizer([b[0] for b in batch], padding=True)
    input_ids = torch.LongTensor(r['input_ids'])
    attention_mask = torch.LongTensor(r['attention_mask'])
    return input_ids, attention_mask, indexs


dldx = []
for d in dlx:
    ds = MyDataSet(d)
    dld = torch.utils.data.DataLoader(
        ds,
        batch_size=batch_size,
        shuffle = True,
        num_workers=data_workers,
        collate_fn=the_collate_fn,
    )
    dldx.append(dld)

In [8]:
from transformers import AdamW, get_linear_schedule_with_warmup

t_total = len(data_list) // gradient_accumulation_steps * max_train_epochs + 1
num_warmup_steps = int(warmup_proportion * t_total)

print('warmup steps : %d' % num_warmup_steps)

no_decay = ['bias', 'LayerNorm.weight'] # no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
param_optimizer = list(model.named_parameters())
optimizer_grouped_parameters = [
    {'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],'weight_decay': weight_decay},
    {'params':[p for n, p in param_optimizer if any(nd in n for nd in no_decay)],'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)

warmup steps : 167510


In [9]:
loss_list = []
for e in range(max_train_epochs):
    print(e)
    loss_sum = 0
    c = 0
    xxx = [x.__iter__() for x in dldx]
    j = 0
    for i in tqdm(range((len(data_list)//batch_size) + 5)):
        if len(xxx) == 0:
            break
        j = j % len(xxx)
        try:
            batch = xxx[j].__next__()
        except StopIteration:
            xxx.pop(j)
            continue
        j += 1
        input_ids, attention_mask, indexs = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss, logits = outputs[:2]
        loss_sum += loss.item()
        c += 1
        loss.backward()
        optimizer.step()
        scheduler.step() 
        optimizer.zero_grad()
    print_cases()
    print(loss_sum / c)
    loss_list.append(loss_sum / c)

0


  0%|          | 37/34903 [00:07<2:01:05,  4.80it/s]


KeyboardInterrupt: 

In [10]:
torch.save(model.state_dict(), 'GPT2_model_parameter.pkl')